# LLM을 활용하여 AI Agent 만들기 (1) StructuredOutputParser을 통한 응답값 구조화

#### Reference : [how-to-use-structuredoutputparser-with-langchain](https://medium.com/@meta_heuristic/how-to-use-structuredoutputparser-with-langchain-6caaa486830)

#### 환경설정

langchain은 LLM을 활용한 어플리케이션 개발 시 사용되는 프레임워크입니다. langchain_openai는 openAI와 연동을 위해 별도로 설치해야 합니다.

In [6]:
%%capture
!pip install langchain langchain_openai

#### OpenAI API KEY 발급받기

<img src="../resources/assets/openai-platform.png" width="30%">

[OpenAI Platform:api-key](https://platform.openai.com/api-keys)에서 API key를 발급받을 수 있습니다.

In [7]:
import getpass
from langchain_openai import ChatOpenAI

In [24]:
api_key = getpass.getpass("openai api-key:")
llm = ChatOpenAI(api_key=api_key)

openai api-key: ········


### LLM을 호출하기

프로그래밍 관점에서 보았을 때, LLM은 텍스트 메시지를 입력받아 텍스트 메시지를 출력하는 함수입니다. 

In [36]:
llm.invoke("안녕")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f691dea-0018-42f8-a7c6-0adda27c8c25-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

LLM에서 메시지 타입은 크게 3가지 유형으로 나뉩니다.

| 유형 | 역할 | 설명 |
| --- | --- | --- |
| SystemMessage | 대화의 맥락을 설정하고, 모델의 행동 지침을 제공 | 모델에게 특정 역할을 부여하거나, 대화의 규칙을 설정하는 데 사용  |
| HumanMessage | 사용자가 모델에게 전달하는 입력 | 사용자가 모델에게 질문을 하거나 요청을 전달하는 데 사용 |
| AIMessage | 모델이 생성한 응답 | 모델이 사용자에게 응답을 제공하는 데 사용 |

아래와 같이 AI에게 특정한 역할과 행동 지침을 부여할 수 있습니다.

In [43]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [44]:
system_message = SystemMessage(content="당신은 유저의 엄격한 도덕 선생님입니다. 유저가 높임말을 쓰도록 훈육해 주세요")

input_message = HumanMessage(content="안녕")

llm.invoke([
    system_message,
    input_message
])

AIMessage(content="안녕하세요. 높임말을 사용해 주시기 바랍니다. 예를 들어 '안녕하세요'라고 쓰시면 좋겠습니다. 감사합니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 64, 'total_tokens': 117, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a9d6a26a-bc8f-4d7f-9b19-de6b5e509303-0', usage_metadata={'input_tokens': 64, 'output_tokens': 53, 'total_tokens': 117})

이렇듯, 같은 인삿말이라도 system_message이 어떻게 정했느냐에 따라, LLM은 서로 다르게 동작합니다.

In [46]:
system_message = SystemMessage(content="당신은 유저의 동갑내기 친구야. 유저에게 반갑게 화답해줘.")

input_message = HumanMessage(content="안녕")

llm.invoke([
    system_message,
    input_message
])

AIMessage(content='안녕! 반가워 😊 함께 재미있는 대화를 나눠보자! 어떤 이야기를 나누고 싶어?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 52, 'total_tokens': 101, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16c0125f-77ba-4c2d-9d5a-8a59c961011b-0', usage_metadata={'input_tokens': 52, 'output_tokens': 49, 'total_tokens': 101})

### LLM의 어려움: 다루기 어려운 텍스트

자연어는 인간이 이해하기 쉬운 형태이지만, 프로그래밍을 작성하는 입장에서는 다루기가 어렵습니다. 예를 들어 단순한 수학 계산을 요청하는 상황을 봅시다.

In [62]:
input_message = HumanMessage(content="x=1, y=2일 때, 2 * (3 * x + y * 5)는 얼마일까요?")

In [63]:
output0 = llm.invoke([input_message])
output0.content

'주어진 식을 계산하면 다음과 같습니다:\n\n2*(3 * x + y * 5) = 2*(3*1 + 2*5) = 2*(3 + 10) = 2*13 = 26\n\n따라서, 2*(3 * x + y * 5)의 값은 26입니다.'

In [64]:
output1 = llm.invoke([input_message])
output1.content

'2*(3 * x + y * 5) = 2*(3 * 1 + 2 * 5) = 2*(3 + 10) = 2*13 = 26\n\n따라서 2*(3 * x + y * 5)는 26입니다.'

이렇게 LLM은 다양한 형태로 응답합니다. 이런 게 다양한 형태의 텍스트에서 우리가 원하는 정답을 도출하는 것은 매우 까다롭습니다.  

### 응답값을 구조화하기

예를들어, chatGPT가 아래와 같이 응답한다면 어떨까요? 

````json
{
   "result": 26,
   "description":"2*(3 * x + y * 5) = 2*(3*1 + 2*5) = 2*(3 + 10) = 2*13 = 26\n\n따라서, 2*(3 * x + y * 5)의 값은 26입니다." 
}
````

우리는 훨씬 더 편하게 값을 처리할 수 있을 것입니다. 이렇게 출력값을 나오도록 `SystemMessage`을 통해 LLM에게 알려주면 됩니다.

````
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"result": float  // The numerical result of the calculation
	"description": string  //description of the calculation process
}
```
````

In [103]:
instruction = (
'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n'
'```json\n'
'{\n'
'	"description": string // description of the calculation process \n'
'	"result": float   // The numerical result of the calculation \n'    
"}\n"
"```\n"
)

output = llm.invoke([
    SystemMessage(instruction),
    input_message]
)
print(output.content)

```json
{
	"description": "Calculate the result of the expression 2*(3*x + y*5) with x=1 and y=2",
	"result": 28.0
}
```


이렇게 훨씬 구조화되어서 응답값이 반환됩니다. `Langchain`에서는 이런 작업들을 도와줄 OutputParser가 존재합니다. 

In [106]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="result", description="The numerical result of the calculation", type='float'),
    ResponseSchema(name="description", description="description of the calculation process", type='string'),
]

output_parser = StructuredOutputParser(response_schemas=response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='result', description='The numerical result of the calculation', type='float'), ResponseSchema(name='description', description='description of the calculation process', type='string')])

파서에서 원하는 형태의 응답값을 받기 위한 지침은 아래와 같이 생성할 수 있습니다.

In [107]:
inst = output_parser.get_format_instructions()
print(inst)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"result": float  // The numerical result of the calculation
	"description": string  // description of the calculation process
}
```


In [108]:
output = llm.invoke([SystemMessage(inst), input_message])
print(output.content)

```json
{
	"result": 32.0,
	"description": "Calculate 3 * x = 3 * 1 = 3 and y * 5 = 2 * 5 = 10. Then, calculate 2 * (3 + 10) = 2 * 13 = 26."
}
```


In [109]:
r = output_parser.invoke(output.content)
r

{'result': 32.0,
 'description': 'Calculate 3 * x = 3 * 1 = 3 and y * 5 = 2 * 5 = 10. Then, calculate 2 * (3 + 10) = 2 * 13 = 26.'}

훨씬 더 구조화된 답변을 받을 수 있습니다. 

In [110]:
r['result'], r['description']

(32.0,
 'Calculate 3 * x = 3 * 1 = 3 and y * 5 = 2 * 5 = 10. Then, calculate 2 * (3 + 10) = 2 * 13 = 26.')